# Working Notebook

__Phase 3 Project - Chicago Traffic Crash Classification__

__Business Understanding__

__Data Understanding and Preparation__

Load dependencies

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV,\
cross_val_score, RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

Import data files

In [2]:
crashes = pd.read_csv('Chicago-Crashes/data/Traffic_Crashes_-_Crashes.csv')
people = pd.read_csv('Chicago-Crashes/data/Traffic_Crashes_-_People.csv')
vehicles = pd.read_csv('Chicago-Crashes/data/Traffic_Crashes_-_Vehicles.csv')

/Users/meaghanross/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/meaghanross/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (12,13,20,21,22,23,24,25,26,27,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/meaghanross/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (19,21,40,41,42,44,48,49,50,53,55,58,59,61,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541142 entries, 0 to 541141
Data columns (total 49 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                541142 non-null  object 
 1   RD_NO                          536550 non-null  object 
 2   CRASH_DATE_EST_I               41029 non-null   object 
 3   CRASH_DATE                     541142 non-null  object 
 4   POSTED_SPEED_LIMIT             541142 non-null  int64  
 5   TRAFFIC_CONTROL_DEVICE         541142 non-null  object 
 6   DEVICE_CONDITION               541142 non-null  object 
 7   WEATHER_CONDITION              541142 non-null  object 
 8   LIGHTING_CONDITION             541142 non-null  object 
 9   FIRST_CRASH_TYPE               541142 non-null  object 
 10  TRAFFICWAY_TYPE                541142 non-null  object 
 11  LANE_CNT                       198968 non-null  float64
 12  ALIGNMENT                     

In [4]:
people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1195747 entries, 0 to 1195746
Data columns (total 30 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   PERSON_ID              1195747 non-null  object 
 1   PERSON_TYPE            1195747 non-null  object 
 2   CRASH_RECORD_ID        1195747 non-null  object 
 3   RD_NO                  1185613 non-null  object 
 4   VEHICLE_ID             1172146 non-null  float64
 5   CRASH_DATE             1195747 non-null  object 
 6   SEAT_NO                244512 non-null   float64
 7   CITY                   879728 non-null   object 
 8   STATE                  890189 non-null   object 
 9   ZIPCODE                803192 non-null   object 
 10  SEX                    1177665 non-null  object 
 11  AGE                    852450 non-null   float64
 12  DRIVERS_LICENSE_STATE  705542 non-null   object 
 13  DRIVERS_LICENSE_CLASS  608027 non-null   object 
 14  SAFETY_EQUIPMENT  

In [5]:
perc = 80.0 # Like N %
min_count =  int(((100-perc)/100)*crashes.shape[0] + 1)
crashes_mod = crashes.dropna(axis=1, 
                            thresh=min_count)

In [6]:
perc = 80.0 # Like N %
min_count =  int(((100-perc)/100)*people.shape[0] + 1)
people_mod = people.dropna(axis=1, 
                            thresh=min_count)

In [7]:
perc = 80.0 # Like N %
min_count =  int(((100-perc)/100)*vehicles.shape[0] + 1)
vehicles_mod = vehicles.dropna(axis=1, 
                            thresh=min_count)

In [8]:
vehicles_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1107267 entries, 0 to 1107266
Data columns (total 19 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   CRASH_UNIT_ID        1107267 non-null  int64  
 1   CRASH_RECORD_ID      1107267 non-null  object 
 2   RD_NO                1097786 non-null  object 
 3   CRASH_DATE           1107267 non-null  object 
 4   UNIT_NO              1107267 non-null  int64  
 5   UNIT_TYPE            1105670 non-null  object 
 6   VEHICLE_ID           1081934 non-null  float64
 7   MAKE                 1081929 non-null  object 
 8   MODEL                1081786 non-null  object 
 9   LIC_PLATE_STATE      987487 non-null   object 
 10  VEHICLE_YEAR         905696 non-null   float64
 11  VEHICLE_DEFECT       1081934 non-null  object 
 12  VEHICLE_TYPE         1081934 non-null  object 
 13  VEHICLE_USE          1081934 non-null  object 
 14  TRAVEL_DIRECTION     1081934 non-null  object 
 15

In [9]:
vehicles_mod.drop(['RD_NO', 'CRASH_DATE', 'UNIT_NO', 'AREA_01_I'], axis =1, inplace=True)

/Users/meaghanross/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [10]:
vehicles_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1107267 entries, 0 to 1107266
Data columns (total 15 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   CRASH_UNIT_ID        1107267 non-null  int64  
 1   CRASH_RECORD_ID      1107267 non-null  object 
 2   UNIT_TYPE            1105670 non-null  object 
 3   VEHICLE_ID           1081934 non-null  float64
 4   MAKE                 1081929 non-null  object 
 5   MODEL                1081786 non-null  object 
 6   LIC_PLATE_STATE      987487 non-null   object 
 7   VEHICLE_YEAR         905696 non-null   float64
 8   VEHICLE_DEFECT       1081934 non-null  object 
 9   VEHICLE_TYPE         1081934 non-null  object 
 10  VEHICLE_USE          1081934 non-null  object 
 11  TRAVEL_DIRECTION     1081934 non-null  object 
 12  MANEUVER             1081934 non-null  object 
 13  OCCUPANT_CNT         1081934 non-null  float64
 14  FIRST_CONTACT_POINT  1073691 non-null  object 
dty

In [11]:
crashes_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541142 entries, 0 to 541141
Data columns (total 41 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                541142 non-null  object 
 1   RD_NO                          536550 non-null  object 
 2   CRASH_DATE                     541142 non-null  object 
 3   POSTED_SPEED_LIMIT             541142 non-null  int64  
 4   TRAFFIC_CONTROL_DEVICE         541142 non-null  object 
 5   DEVICE_CONDITION               541142 non-null  object 
 6   WEATHER_CONDITION              541142 non-null  object 
 7   LIGHTING_CONDITION             541142 non-null  object 
 8   FIRST_CRASH_TYPE               541142 non-null  object 
 9   TRAFFICWAY_TYPE                541142 non-null  object 
 10  LANE_CNT                       198968 non-null  float64
 11  ALIGNMENT                      541142 non-null  object 
 12  ROADWAY_SURFACE_COND          

In [12]:
crashes_mod.drop(['RD_NO','REPORT_TYPE', 'DATE_POLICE_NOTIFIED', 'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME'], axis =1, inplace=True)

In [13]:
people_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1195747 entries, 0 to 1195746
Data columns (total 22 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   PERSON_ID              1195747 non-null  object 
 1   PERSON_TYPE            1195747 non-null  object 
 2   CRASH_RECORD_ID        1195747 non-null  object 
 3   RD_NO                  1185613 non-null  object 
 4   VEHICLE_ID             1172146 non-null  float64
 5   CRASH_DATE             1195747 non-null  object 
 6   SEAT_NO                244512 non-null   float64
 7   CITY                   879728 non-null   object 
 8   STATE                  890189 non-null   object 
 9   ZIPCODE                803192 non-null   object 
 10  SEX                    1177665 non-null  object 
 11  AGE                    852450 non-null   float64
 12  DRIVERS_LICENSE_STATE  705542 non-null   object 
 13  DRIVERS_LICENSE_CLASS  608027 non-null   object 
 14  SAFETY_EQUIPMENT  

In [14]:
people_mod.drop(['CRASH_DATE', 'CITY', 'ZIPCODE','RD_NO'], axis =1, inplace=True)

In [15]:
people_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1195747 entries, 0 to 1195746
Data columns (total 18 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   PERSON_ID              1195747 non-null  object 
 1   PERSON_TYPE            1195747 non-null  object 
 2   CRASH_RECORD_ID        1195747 non-null  object 
 3   VEHICLE_ID             1172146 non-null  float64
 4   SEAT_NO                244512 non-null   float64
 5   STATE                  890189 non-null   object 
 6   SEX                    1177665 non-null  object 
 7   AGE                    852450 non-null   float64
 8   DRIVERS_LICENSE_STATE  705542 non-null   object 
 9   DRIVERS_LICENSE_CLASS  608027 non-null   object 
 10  SAFETY_EQUIPMENT       1192258 non-null  object 
 11  AIRBAG_DEPLOYED        1173141 non-null  object 
 12  EJECTION               1181206 non-null  object 
 13  INJURY_CLASSIFICATION  1195154 non-null  object 
 14  DRIVER_ACTION     

In [16]:
people_mod.head()

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,VEHICLE_ID,SEAT_NO,STATE,SEX,AGE,DRIVERS_LICENSE_STATE,DRIVERS_LICENSE_CLASS,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,EJECTION,INJURY_CLASSIFICATION,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,834816.0,NaN,IL,M,25.0,IL,D,NONE PRESENT,DEPLOYMENT UNKNOWN,NONE,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,827212.0,NaN,IL,M,37.0,IL,NaN,SAFETY BELT USED,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,NONE,NOT OBSCURED,NORMAL,TEST NOT OFFERED
2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,9579.0,NaN,NaN,X,NaN,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NONE,NO INDICATION OF INJURY,IMPROPER BACKING,UNKNOWN,UNKNOWN,TEST NOT OFFERED
3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,9598.0,NaN,NaN,X,NaN,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED
4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,9600.0,NaN,NaN,X,NaN,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,UNKNOWN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED


In [17]:
vehicles_mod.head()

,CRASH_UNIT_ID,CRASH_RECORD_ID,UNIT_TYPE,VEHICLE_ID,MAKE,MODEL,LIC_PLATE_STATE,VEHICLE_YEAR,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,FIRST_CONTACT_POINT
0,829999,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,DRIVER,796949.0,INFINITI,UNKNOWN,IL,2017.0,NONE,PASSENGER,PERSONAL,N,STRAIGHT AHEAD,1.0,FRONT
1,749947,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,DRIVER,834816.0,HONDA,CIVIC,IL,2016.0,UNKNOWN,PASSENGER,PERSONAL,N,STRAIGHT AHEAD,1.0,FRONT
2,749949,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,PARKED,834819.0,TOYOTA,YARIS,IL,2010.0,NONE,UNKNOWN/NA,PERSONAL,N,PARKED,0.0,ROOF
3,749950,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,PARKED,834817.0,GENERAL MOTORS CORPORATION (GMC),SIERRA,IL,2008.0,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,N,PARKED,0.0,ROOF
4,871921,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,DRIVER,827212.0,BUICK,ENCORE,IL,NaN,NONE,PASSENGER,PERSONAL,W,STRAIGHT AHEAD,1.0,FRONT-RIGHT


In [18]:
crashes_mod.head()

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,ONE-WAY,NaN,...,0.0,0.0,3.0,0.0,17,4,7,41.919664,-87.773288,POINT (-87.773287883007 41.919663832993)
1,009e9e67203442370272e1a13d6ee51a4155dac65e583d...,06/30/2017 04:00:00 PM,35,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,4.0,...,0.0,0.0,3.0,0.0,16,6,6,41.741804,-87.740954,POINT (-87.740953581987 41.741803598989)
2,ee9283eff3a55ac50ee58f3d9528ce1d689b1c4180b4c4...,07/10/2020 10:25:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,FOUR WAY,NaN,...,0.0,0.0,3.0,0.0,10,6,7,41.773456,-87.585022,POINT (-87.585022352022 41.773455972008)
3,f8960f698e870ebdc60b521b2a141a5395556bc3704191...,07/11/2020 01:00:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),NaN,...,0.0,0.0,3.0,0.0,1,7,7,41.802119,-87.622115,POINT (-87.622114914961 41.802118543011)
4,8eaa2678d1a127804ee9b8c35ddf7d63d913c14eda61d6...,07/08/2020 02:00:00 PM,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DRIVEWAY,NaN,...,0.0,0.0,1.0,0.0,14,4,7,NaN,NaN,NaN


__First Simple Model__

In [19]:
crashes_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541142 entries, 0 to 541141
Data columns (total 35 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                541142 non-null  object 
 1   CRASH_DATE                     541142 non-null  object 
 2   POSTED_SPEED_LIMIT             541142 non-null  int64  
 3   TRAFFIC_CONTROL_DEVICE         541142 non-null  object 
 4   DEVICE_CONDITION               541142 non-null  object 
 5   WEATHER_CONDITION              541142 non-null  object 
 6   LIGHTING_CONDITION             541142 non-null  object 
 7   FIRST_CRASH_TYPE               541142 non-null  object 
 8   TRAFFICWAY_TYPE                541142 non-null  object 
 9   LANE_CNT                       198968 non-null  float64
 10  ALIGNMENT                      541142 non-null  object 
 11  ROADWAY_SURFACE_COND           541142 non-null  object 
 12  ROAD_DEFECT                   

In [20]:
crashes_mod.MOST_SEVERE_INJURY.value_counts()

NO INDICATION OF INJURY     468994
NONINCAPACITATING INJURY     39675
REPORTED, NOT EVIDENT        21763
INCAPACITATING INJURY         9040
FATAL                          547
Name: MOST_SEVERE_INJURY, dtype: int64

Set up target variable:
 - 0: NO INDICATION OF INJURY, NONINCAPACITATING INJURY, REPORTED, NOT EVIDENT
 - 1: INCAPACITATING INJURY, FATAL
 

In [21]:
crashes_mod['TARGET']= crashes_mod['MOST_SEVERE_INJURY'].map({'NO INDICATION OF INJURY': 0,
                                                              'NONINCAPACITATING INJURY': 0,
                                                              'REPORTED, NOT EVIDENT': 0,
                                                              'INCAPACITATING INJURY': 1,
                                                              'FATAL': 1})

<ipython-input-21-c9d91aeaf547>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crashes_mod['TARGET']= crashes_mod['MOST_SEVERE_INJURY'].map({'NO INDICATION OF INJURY': 0,


In [22]:

crashes_mod.TARGET.fillna(0, inplace=True)

/Users/meaghanross/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [23]:
crashes_mod.TARGET.isna().sum()

0

In [24]:
from sklearn.model_selection import train_test_split
X = crashes_mod.drop(['MOST_SEVERE_INJURY','INJURIES_TOTAL', 'INJURIES_FATAL','INJURIES_INCAPACITATING',
                      'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT',
                      'INJURIES_NO_INDICATION', 'TARGET', 'CRASH_RECORD_ID', 'CRASH_DATE', 'LATITUDE', 
                      'LONGITUDE','LOCATION'], axis=1)
y = crashes_mod['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [25]:
X_train_nums = X_train.select_dtypes(include=['float64', 'int64'])
X_train_cat = X_train.select_dtypes('object')

In [49]:
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
#ss = StandardScaler()

X_train_impute = imputer.fit(X_train_nums)
nums_df = pd.DataFrame(imputer.transform(X_train_nums),
                      index=X_train_nums.index)

In [50]:
ss =StandardScaler()
nums_df_scaled = pd.DataFrame(ss.fit_transform(nums_df), index =X_train_nums.index)

In [51]:
nums_df_scaled.head()

,0,1,2,3,4,5,6,7
84547,0.263826,-0.002435,-0.289998,-0.076151,0.0,-0.587558,-0.065049,-1.072218
21885,1.042026,-0.002435,1.404821,-0.076151,0.0,0.860698,0.945271,0.409510
456760,0.263826,-0.002435,-0.164194,-0.076151,0.0,-2.035813,0.440111,0.705856
440266,0.263826,-0.002435,-0.586838,2.149580,0.0,0.317602,0.945271,1.594894
377470,-4.405368,-0.003643,-0.571289,-0.076151,0.0,-1.130654,-0.065049,1.298548


In [52]:
ohe = OneHotEncoder(
    drop='first',
    sparse=False)

dums = ohe.fit_transform(X_train_cat)
dums_df = pd.DataFrame(dums,
                       columns=ohe.get_feature_names(),
                       index=X_train_cat.index)

In [53]:
X_train_all = pd.concat([nums_df_scaled, dums_df], axis=1)

In [27]:
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy="median")),
    ('ss', StandardScaler())
])
                
categorical_pipeline = Pipeline(steps=[
    ('ohe', OneHotEncoder(drop='first',
                         sparse=False))
])

trans = ColumnTransformer(transformers=[
    ('numerical', numerical_pipeline, X_train_nums.columns),
    ('categorical', categorical_pipeline, X_train_cat.columns)
])

In [28]:
model_pipe = Pipeline(steps=[
    ('trans', trans),
    ('dc', DummyClassifier(strategy='most_frequent'))
])

In [30]:
model_pipe.fit(X_train, y_train)

Pipeline(steps=[('trans',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('ss',
                                                                   StandardScaler())]),
                                                  Index(['POSTED_SPEED_LIMIT', 'LANE_CNT', 'BEAT_OF_OCCURRENCE', 'NUM_UNITS',
       'INJURIES_UNKNOWN', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH'],
      dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('ohe'...
                                                  Index(['TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION', 'WEATHER_CONDITION',
       'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE',
       'A

In [31]:
model_pipe.score(X_train, y_train)

0.9824026255617756

In [32]:
model_pipe.score(X_test, y_test)

0.9819271765001553